In [17]:
import numpy as np
import matlab.engine
import pandas as pd
import scipy.io as sio

from keras.models import load_model
from sklearn.metrics import r2_score, mean_squared_error
import tensorflow as tf

import sys
sys.path.insert(0, '../')

from functions.functions import load_data_forGridSearch, feature_scaling_forGridSearch, load_object


In [8]:
# load metrolab data for testing

data = pd.read_csv("metrolab_unified_units.csv",
                   skiprows=1, header=None)
print("Total data shape is ", data.shape)

Total data shape is  (214974, 14)


In [9]:
X_test_metrolab = data.values[:, :-3]
y_test_metrolab = data.values[:, -3:]
print("{} samples are finally selected for testing".format(len(X_test)))

214974 samples are finally selected for testing


In [10]:
# load sensor grid training data to feature scaling metrolab data
X_train, _ = load_data_forGridSearch("../../Data", "train")

# perform feature scaling
_, X_test_metrolab_transformed = feature_scaling_forGridSearch(X_train, X_test_metrolab)


# print(np.max(X_train_transformed, axis=0))
# print(np.min(X_train_transformed, axis=0))
# print("*"*10)
# print(np.max(X_test_transformed, axis=0))
# print(np.min(X_test_transformed, axis=0))

# print("\nsensor grid and metrolab have different sensor location range and (slightly) current range.")


In [11]:
def evaluate_generic_metrics(labels, predictions):
    
    # label_norm = np.sqrt(np.sum(labels**2, axis=1))
    # prediction_norm = np.sqrt(np.sum(predictions**2, axis=1))
    label_norm = [np.linalg.norm(y) for y in labels]
    prediction_norm = [np.linalg.norm(y) for y in predictions]
    
    # R^2
    r2_c = r2_score(y_true=labels, y_pred=predictions, multioutput='raw_values')
    r2 = r2_score(y_true=labels, y_pred=predictions)
    r2_norm = r2_score(y_true=label_norm, y_pred=prediction_norm)

    # Root mean squared error
    rmse_c = np.sqrt(mean_squared_error(y_true=labels, y_pred=predictions, multioutput='raw_values'))
    rmse = np.sqrt(mean_squared_error(y_true=labels, y_pred=predictions))
    rmse_norm = np.sqrt(mean_squared_error(y_true=label_norm, y_pred=prediction_norm))

    return {"R2_x": round(r2_c[0], 2),
            "R2_y": round(r2_c[1], 2),
            "R2_z": round(r2_c[2], 2),
            "R2": round(r2, 2),
            "R2_norm": round(r2_norm, 2),
            "RMSE_x_mT": round(rmse_c[0]*1000, 2),
            "RMSE_y_mT": round(rmse_c[1]*1000, 2),
            "RMSE_z_mT": round(rmse_c[2]*1000, 2),
            "RMSE_mT": round(rmse*1000, 2),
            "RMSE_norm_mT": round(rmse_norm*1000,2)}

## Linear baseline model (recalibrated)

In [21]:
# load matlab predictions
recalibrated_y_pred = sio.loadmat('../baseline_model/magnetic_model/CalibrateSystem_CardioMag_retrained_w_SensorGrid/metrolab_y_pred.mat')['fieldStrength']
recalibrated_y_pred = recalibrated_y_pred[:,0:3]
assert recalibrated_y_pred.shape == y_test.shape, "Predictions for testing set do not have the same shape as the labels"

In [22]:
baseline_results_recalibrated = evaluate_generic_metrics(y_test_metrolab, recalibrated_y_pred)
baseline_results_recalibrated

{'R2_x': 0.88,
 'R2_y': 0.83,
 'R2_z': 0.82,
 'R2': 0.84,
 'R2_norm': 0.62,
 'RMSE_x_mT': 12.3,
 'RMSE_y_mT': 12.92,
 'RMSE_z_mT': 11.22,
 'RMSE_mT': 12.17,
 'RMSE_norm_mT': 18.31}

## Random forest model

In [12]:
# load model
grid_RF = load_object("../../Models/RF/GridSearch_RFmulti.pkl")
# model_rf = grid_RF.best_estimator_

/Users/ruoxi/anaconda3/envs/DL/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.19.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ruoxi/anaconda3/envs/DL/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.19.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ruoxi/anaconda3/envs/DL/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator GridSearchCV from version 0.19.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [13]:
rf_y_pred = grid_RF.predict(X_test_metrolab_transformed)

In [16]:
rf_results = evaluate_generic_metrics(y_test_metrolab, rf_y_pred)
rf_results

{'R2_x': 0.85,
 'R2_y': 0.85,
 'R2_z': 0.77,
 'R2': 0.82,
 'R2_norm': 0.79,
 'RMSE_x_mT': 13.84,
 'RMSE_y_mT': 12.13,
 'RMSE_z_mT': 12.55,
 'RMSE_mT': 12.86,
 'RMSE_norm_mT': 13.63}

## ANN model

In [18]:
def R2_score(y_true, y_pred):
    numerator = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
    denominator = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
    R2 = 1. - tf.div(numerator, denominator)
    R2_mean = tf.reduce_mean(R2)
    return R2_mean

model = load_model('../../Models/ANN/model.hdf5', custom_objects={'R2_score': R2_score})
ann_y_pred = model.predict(X_test_metrolab_transformed)
ann_results = evaluate_generic_metrics(y_test_metrolab, ann_y_pred)
ann_results

{'R2_x': 0.99,
 'R2_y': 0.99,
 'R2_z': 0.98,
 'R2': 0.99,
 'R2_norm': 0.99,
 'RMSE_x_mT': 3.46,
 'RMSE_y_mT': 2.86,
 'RMSE_z_mT': 3.53,
 'RMSE_mT': 3.3,
 'RMSE_norm_mT': 3.35}

## Overall results

In [24]:
baseline_results_recalibrated['model'] = 'linear baseline model (recalibrated)'
rf_results['model'] = 'random forest'
ann_results['model'] = 'ANN'

results = pd.DataFrame.from_dict([baseline_results_recalibrated, rf_results, ann_results])
results[['model', 'R2_x','R2_y', 'R2_z','R2_norm', 'RMSE_x_mT', 'RMSE_y_mT', 'RMSE_z_mT', 'RMSE_norm_mT']]

,model,R2_x,R2_y,R2_z,R2_norm,RMSE_x_mT,RMSE_y_mT,RMSE_z_mT,RMSE_norm_mT
0,linear baseline model (recalibrated),0.88,0.83,0.82,0.62,12.30,12.92,11.22,18.31
1,random forest,0.85,0.85,0.77,0.79,13.84,12.13,12.55,13.63
2,ANN,0.99,0.99,0.98,0.99,3.46,2.86,3.53,3.35
